In [1]:
import torch
import config
import utils
import glob
import numpy as np
import cv2
from pathlib import Path

In [2]:
def draw_bboxes(img, preds, thre, class_colors, save_fname):
    preds = [{k: v.to('cpu') for k,v in t.items()} for t in preds]

    if len(preds[0]['boxes']) != 0:
        boxes = preds[0]['boxes'].data.numpy()
        scores = preds[0]['scores'].data.numpy()
        print(f"boxes={boxes}, scores = {scores}")
        
        boxes = boxes[scores >= thre].astype(np.int32)
        pred_classes = [i for i in preds[0]['labels'].cpu().numpy() ]

        for j, box in enumerate(boxes):
            color = class_colors[pred_classes[j]]
            cv2.rectangle(img,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        color, 2)

        # save the image
        cv2.imwrite(save_fname, img)

        
def inference_1img(model, img_name, device, thre, class_colors):
    in_img = cv2.imread(img_name)

    # convert to tensor
    img = cv2.cvtColor(in_img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img /= 255.0
    img = np.transpose(img, (2,0,1)) # HWC -> CHW
    img = torch.tensor(img, dtype=torch.float).to(device)
    img = torch.unsqueeze(img,0) # add batch dim

    # run inference
    with torch.no_grad():
        preds = model(img)
    print(f"inference on {img_name} done.")

    save_fname = str(Path(config.result_img_dir) / Path(img_name).name)
    draw_bboxes(in_img, preds, thre, class_colors, save_fname)    


In [3]:
Path(config.result_img_dir).mkdir(parents=True, exist_ok=True)
    
device = torch.device("cpu")
saved_name = './result/small'
checkpoint = torch.load(saved_name, map_location=device)
model = utils.get_model_object_detector(config.num_classes)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device).eval()

test_dir = config.test_data_dir
img_format = config.test_img_format
test_imgs = glob.glob(f"{test_dir}/*.{img_format}")

class_colors = np.random.uniform(0, 255, size=(config.num_classes, 3))

for i in range(len(test_imgs)):
    img_name = test_imgs[i]
    inference_1img(model, img_name, device, config.detection_threshold, class_colors)

d:\Source\AI&ML\FML\CSCN8010\venv\pytorch_cpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Source\AI&ML\FML\CSCN8010\venv\pytorch_cpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


inference on data/test\Untitled.png done.
boxes=[[8.2468219e+02 5.6749188e+02 9.2961841e+02 6.2131970e+02]
 [1.0978018e+03 5.3697491e+02 1.2118662e+03 5.9579071e+02]
 [7.0221967e+02 5.2832184e+02 8.0311743e+02 5.8425348e+02]
 [2.5777090e+02 3.4198254e+02 2.7844568e+02 3.8100613e+02]
 [1.1235582e+03 4.8005771e+02 1.2692780e+03 5.6249249e+02]
 [1.4521315e+03 4.8328381e+02 1.5710261e+03 5.3130042e+02]
 [1.6338389e+03 3.9374347e+02 1.6543824e+03 4.3626874e+02]
 [1.6504432e+03 3.4871237e+02 1.6720729e+03 3.7781552e+02]
 [4.5979581e+02 4.2576825e+02 6.6936694e+02 5.2086755e+02]
 [1.7632323e+03 4.5467047e+02 1.7822448e+03 4.9398419e+02]
 [1.2822169e+03 4.2700455e+02 1.4542227e+03 5.2422424e+02]
 [1.8799055e+03 5.5144000e+02 1.9111193e+03 5.9792114e+02]
 [1.7242802e+03 4.1835968e+02 1.7474592e+03 4.6187683e+02]
 [1.4733444e+02 4.4414102e+02 1.7044106e+02 4.7993378e+02]
 [9.2489784e+01 4.7170883e+02 1.1981610e+02 5.1280664e+02]
 [2.2661777e+02 5.3793323e+02 2.5673843e+02 5.6915680e+02]
 [2.9027

In [4]:
# import required libraries
from vidgear.gears import CamGear
import cv2

# set desired quality as 720p
options = {"STREAM_RESOLUTION": "720p"}

# Add any desire Video URL as input source
# for e.g hhttps://www.twitch.tv/shroud
# and enable Stream Mode (`stream_mode = True`)
stream = CamGear(
    source="https://www.twitch.tv/firstcanada",
    stream_mode=True,
    logging=True,
    **options
).start()

# loop over
while True:

    # read frames from stream
    frame = stream.read()

    # check for frame if Nonetype
    if frame is None:
        break

    # {do something with the frame here}

    # Show output window
    cv2.imshow("Output", frame)

    # check for 'q' key if pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# close output window
cv2.destroyAllWindows()

# safely close video stream
stream.stop()


15:38:28 ::    Helper     :: warning  :: GStreamer not found!
15:38:28 ::    Helper     ::  debug   :: Selecting `720p` resolution for streams.
15:38:34 ::    CamGear    ::  debug   :: Enabling Threaded Queue Mode for the current video source!


RuntimeError: [CamGear:ERROR] :: Source is invalid, CamGear failed to initialize stream on this source!